# Assessment of google covid datasets

This notebook assesses the selected covid datasets of google.

In detail the datasets are:

* an index file containing information to the keys used in the datasets
* the epidemiology dataset
* the mobility dataset
* the search trends dataset

## imports

In [1]:
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1609146570094_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## define bucket

In [1]:
#FIXME
input_bucket = 's3://{YOUR_BUCKET}/stage0_raw'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1609146570094_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Google open covid data

## Index file

In [2]:
#read in index file
gl_index_file = '/google/index.csv'
df_index = spark.read.format('csv').options(header='true').load(input_bucket + gl_index_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# show a few records
df_index.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-----------+------------+--------------------+---------------+---------------+---------------+---------------+-------------+-------------+--------------+--------------+-----------------+
|   key|wikidata|datacommons|country_code|        country_name|subregion1_code|subregion1_name|subregion2_code|subregion2_name|locality_code|locality_name|3166-1-alpha-2|3166-1-alpha-3|aggregation_level|
+------+--------+-----------+------------+--------------------+---------------+---------------+---------------+---------------+-------------+-------------+--------------+--------------+-----------------+
|    AD|    Q228|country/AND|          AD|             Andorra|           null|           null|           null|           null|         null|         null|            AD|           AND|                0|
|    AE|    Q878|country/ARE|          AE|United Arab Emirates|           null|           null|           null|           null|         null|         null|            AE|           ARE

In [9]:
# count the records
df_index.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

21505

In [10]:
# print schema
df_index.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: string (nullable = true)
 |-- wikidata: string (nullable = true)
 |-- datacommons: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- subregion1_code: string (nullable = true)
 |-- subregion1_name: string (nullable = true)
 |-- subregion2_code: string (nullable = true)
 |-- subregion2_name: string (nullable = true)
 |-- locality_code: string (nullable = true)
 |-- locality_name: string (nullable = true)
 |-- 3166-1-alpha-2: string (nullable = true)
 |-- 3166-1-alpha-3: string (nullable = true)
 |-- aggregation_level: string (nullable = true)

In [11]:
# show how detailed the area codes are for which country
df_index.groupBy("country_code").count().orderBy('count', ascending=False).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|country_code|count|
+------------+-----+
|          BR| 5616|
|          US| 3285|
|          MX| 2500|
|          PE| 1901|
|          IL| 1491|
|          ES| 1398|
|          CO| 1156|
|          IN|  773|
|          AR|  545|
|          NL|  368|
|          CL|  363|
|          GB|  197|
|          PH|  131|
|          IT|  125|
|          FR|  118|
|          CA|  106|
|          AT|  104|
|          CZ|   92|
|          RU|   86|
|          BD|   73|
+------------+-----+
only showing top 20 rows

In [12]:
# show the number of entries for the aggregation levels
df_index.groupBy("aggregation_level").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+
|aggregation_level|count|
+-----------------+-----+
|                3|   32|
|                0|  247|
|                1| 1269|
|                2|19957|
+-----------------+-----+

## Epidemiology dataset

In [13]:
#read in epidemiology dataset
gl_epi_file = '/google/epidemiology.csv'
df_epi = spark.read.format('csv').options(header='true').load(input_bucket + gl_epi_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# create view to enable sql
df_epi.createOrReplaceTempView("epi_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# show a few records
df_epi.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-------------+------------+-------------+----------+---------------+--------------+---------------+------------+
|      date|      key|new_confirmed|new_deceased|new_recovered|new_tested|total_confirmed|total_deceased|total_recovered|total_tested|
+----------+---------+-------------+------------+-------------+----------+---------------+--------------+---------------+------------+
|2019-10-23|       CZ|         null|        null|            1|      null|           null|          null|              1|        null|
|2019-10-23|    CZ_32|         null|        null|            1|      null|           null|          null|              1|        null|
|2019-10-23|CZ_32_325|         null|        null|            1|      null|           null|          null|              1|        null|
|2019-12-30|       AE|            0|           0|         null|      null|              0|             0|           null|        null|
|2019-12-30|       AF|            0|           0|      

In [20]:
# count the records
df_epi.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3930988

In [21]:
# print schema
df_epi.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: string (nullable = true)
 |-- key: string (nullable = true)
 |-- new_confirmed: string (nullable = true)
 |-- new_deceased: string (nullable = true)
 |-- new_recovered: string (nullable = true)
 |-- new_tested: string (nullable = true)
 |-- total_confirmed: string (nullable = true)
 |-- total_deceased: string (nullable = true)
 |-- total_recovered: string (nullable = true)
 |-- total_tested: string (nullable = true)

In [22]:
# check how many date entries we have and min and max date
spark.sql("""select count(distinct date), min(date), max(date) from epi_data""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+----------+
|count(DISTINCT date)| min(date)| max(date)|
+--------------------+----------+----------+
|                 354|2019-10-23|2020-12-16|
+--------------------+----------+----------+

### FINDINGS
* convert date to date
* convert metrics to int
* date is not continous => check
* null values in metrics
* join with index

## Mobility dataset

In [23]:
#read in mobility dataset
gl_mob_file = '/google/mobility.csv'
df_mob = spark.read.format('csv').options(header='true').load(input_bucket + gl_mob_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# show some records
df_mob.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---+------------------------------+-----------------------------+--------------+-------------------------+-------------------+--------------------+
|      date|key|mobility_retail_and_recreation|mobility_grocery_and_pharmacy|mobility_parks|mobility_transit_stations|mobility_workplaces|mobility_residential|
+----------+---+------------------------------+-----------------------------+--------------+-------------------------+-------------------+--------------------+
|2020-02-15| AE|                             0|                            4|             5|                        0|                  2|                   1|
|2020-02-15| AF|                            -9|                          -10|             0|                       -2|                -28|                   5|
|2020-02-15| AG|                            -6|                           -1|            -6|                       -1|                  0|                   3|
|2020-02-15| AO|                        

In [29]:
#print schema
df_mob.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: string (nullable = true)
 |-- key: string (nullable = true)
 |-- mobility_retail_and_recreation: string (nullable = true)
 |-- mobility_grocery_and_pharmacy: string (nullable = true)
 |-- mobility_parks: string (nullable = true)
 |-- mobility_transit_stations: string (nullable = true)
 |-- mobility_workplaces: string (nullable = true)
 |-- mobility_residential: string (nullable = true)

In [47]:
#count records
df_mob.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2052120

In [46]:
#show summary
df_mob.summary().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-------+------------------------------+-----------------------------+-------------------+-------------------------+-------------------+--------------------+
|summary|      date|    key|mobility_retail_and_recreation|mobility_grocery_and_pharmacy|     mobility_parks|mobility_transit_stations|mobility_workplaces|mobility_residential|
+-------+----------+-------+------------------------------+-----------------------------+-------------------+-------------------------+-------------------+--------------------+
|  count|   2052120|2052120|                       1341979|                      1270369|             977820|                  1037377|            1963115|             1215295|
|   mean|      null|   null|           -22.695039192118507|          -2.1622087755604866|-10.351330510727946|      -26.249076276030795|-18.636363636363637|   9.532744724531904|
| stddev|      null|   null|            27.663889312490696|           23.357345155031435|  58.03008145922868|      

In [44]:
#create view to enable the use of sql on the dataset
df_mob.createOrReplaceTempView("mob_data")

#check most present keys
spark.sql("""select key, count(*) as c from mob_data group by key order by c desc""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---+
|         key|  c|
+------------+---+
|  NL_LI_0935|301|
|   IN_TR_NTR|301|
|       KE_31|301|
|   IN_MH_SAT|301|
|   IN_OR_ANU|301|
|        AR_H|301|
|          CI|301|
|BR_RJ_330250|301|
|   IN_MH_NNR|301|
|   IN_MN_CHC|301|
|       ZA_MP|301|
|   IN_MP_KHA|301|
|BR_SP_352680|301|
|      MX_TAM|301|
|BR_ES_320130|301|
|       NL_LI|301|
|   IN_KL_PTT|301|
| US_LA_22005|301|
|       BR_BA|301|
| US_MT_30063|301|
+------------+---+
only showing top 20 rows

In [36]:
#select number of present dates and their minimum and maximum
spark.sql("""select count(distinct date), min(date), max(date) from mob_data""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+----------+
|count(DISTINCT date)| min(date)| max(date)|
+--------------------+----------+----------+
|                 301|2020-02-15|2020-12-11|
+--------------------+----------+----------+

In [42]:
# check for null values in key columns
spark.sql("""select count(*) as cnt_key_is_null from mob_data where key is null""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|cnt_key_is_null|
+---------------+
|              0|
+---------------+

In [41]:
# check for null values in date columns
spark.sql("""select count(*) as cnt_date_is_null from mob_data where date is null""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|cnt_date_is_null|
+----------------+
|               0|
+----------------+

### FINDINGS
* convert date to date
* convert metrics to int or double?
* date is not continous => check
* null values in metrics
* transpose metrics?

## Search Trends dataset

In [48]:
#read in search trends dataset
gl_st_file = '/google/google-search-trends.csv'
df_st = spark.read.format('csv').options(header='true').load(input_bucket + gl_st_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
#count the records
df_st.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

957338

In [50]:
#print schema
df_st.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: string (nullable = true)
 |-- key: string (nullable = true)
 |-- search_trends_abdominal_obesity: string (nullable = true)
 |-- search_trends_abdominal_pain: string (nullable = true)
 |-- search_trends_acne: string (nullable = true)
 |-- search_trends_actinic_keratosis: string (nullable = true)
 |-- search_trends_acute_bronchitis: string (nullable = true)
 |-- search_trends_adrenal_crisis: string (nullable = true)
 |-- search_trends_ageusia: string (nullable = true)
 |-- search_trends_alcoholism: string (nullable = true)
 |-- search_trends_allergic_conjunctivitis: string (nullable = true)
 |-- search_trends_allergy: string (nullable = true)
 |-- search_trends_amblyopia: string (nullable = true)
 |-- search_trends_amenorrhea: string (nullable = true)
 |-- search_trends_amnesia: string (nullable = true)
 |-- search_trends_anal_fissure: string (nullable = true)
 |-- search_trends_anaphylaxis: string (nullable = true)
 |-- search_trends_anemia: string (nullable = true)
 |--

In [51]:
# show some records
df_st.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------------------------------+----------------------------+------------------+-------------------------------+------------------------------+----------------------------+---------------------+------------------------+-------------------------------------+---------------------+-----------------------+------------------------+---------------------+--------------------------+-------------------------+--------------------+-----------------------------+------------------------+-------------------------------+---------------------+---------------------+---------------------+---------------------+-------------------+------------------------+-----------------------+---------------------+-------------------------------+----------------------+--------------------+-------------------------+--------------------+----------------------+------------------------------------------------------+------------------------------------+--------------------------------+--------------------

In [52]:
#create temp view to enable the use of sql
df_st.createOrReplaceTempView("st_data")

#check most present keys
spark.sql("""select key, count(*) as c from st_data group by key order by c desc""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---+
|        key|  c|
+-----------+---+
|US_MA_25017|348|
|US_TX_48241|348|
|US_IA_19121|348|
|US_FL_12005|348|
|US_GA_13229|348|
|US_CA_06071|348|
|US_LA_22041|348|
|US_FL_12133|348|
|US_PA_42075|348|
|US_TN_47115|348|
|US_VA_51820|348|
|US_GA_13237|348|
|US_FL_12013|348|
|US_MT_30067|348|
|US_VA_51610|348|
|US_PA_42019|348|
|US_KY_21211|348|
|US_AR_05001|348|
|US_KY_21205|348|
|US_OR_41001|348|
+-----------+---+
only showing top 20 rows

In [53]:
#select number of present dates and their minimum and maximum
spark.sql("""select count(distinct date), min(date), max(date) from st_data""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+----------+
|count(DISTINCT date)| min(date)| max(date)|
+--------------------+----------+----------+
|                 348|2020-01-01|2020-12-13|
+--------------------+----------+----------+

In [54]:
# check for null values in key columns
spark.sql("""select count(*) as cnt_key_is_null from mob_data where key is null""").show()
spark.sql("""select count(*) as cnt_date_is_null from mob_data where date is null""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|cnt_key_is_null|
+---------------+
|              0|
+---------------+

+----------------+
|cnt_date_is_null|
+----------------+
|               0|
+----------------+

### FINDINGS
* convert date to date
* convert metrics to double
* date is not continous => check
* null values in metrics
* transpose metrics